# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 layer, module로 구성되어 있다.

torch.nn name space는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.

pytorch의 모든 모듈은 서브 클래스 nn.Module의 subclass이다.

신경망은 다른 layer로 구성된 모듈이다.

이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보자




In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# gpu 사용 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# define class
신경망 모델을 nn.Module의 하위 클래스로 정의

__init__에서 신경망 계층 초기화

nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module): # nn.Module의 하위 클래스로 정의
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        

In [4]:
# NeuralNetwork의 인스턴스를 생성하고 structure 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 입력 데이터를 전달

이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행한다.
--> model.forward()를 직접 호출하지 마라 

모델에 입력을 호출하면 각 class에 대한 raw 예측값이 있는 10차원 텐서가 반환된다.

raw 예측값을 nn.softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [5]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print("Predicted Class")
print(y_pred)

Predicted Class
tensor([9], device='cuda:0')


# 모델 레이어

FashionMNIST 모델의 레이어를 분해해보자.

이를 설명하기 위해 크기가 28x28인 이미지 3개의 샘플 미니배치를 가져와 네트워크를 통해 전달할 때 어떻게 되는지 확인하자.

In [14]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten

nn.Flatten 레이어를 초기화해서 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환하자.
(dim=0의 미니배치 차원은 유지된다)

In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear

linear 레이어는 저장된 weights와 bias를 사용하여 입력에 linear transformation을 적용하는 모듈이다.

In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU

비선형 activation 함수는 모델의 입력과 출력 사이에 복잡한 mapping을 만든다.

비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고,

신경망이 다양한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [19]:
print("Before ReLU")
print(hidden1)

hidden1 = nn.ReLU()(hidden1)

print("After ReLU")
print(hidden1)

Before ReLU
tensor([[0.0000, 0.4028, 0.0000, 0.5694, 0.0000, 0.0000, 0.3374, 0.3942, 0.5255,
         0.1611, 0.1811, 0.0444, 0.4902, 0.6375, 0.1585, 0.0000, 0.1575, 0.0000,
         0.0000, 0.1211],
        [0.0000, 0.2777, 0.0000, 0.3033, 0.0000, 0.1457, 0.4572, 0.6595, 0.7586,
         0.3265, 0.0000, 0.0000, 0.6714, 0.6085, 0.2807, 0.0544, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2468, 0.3046, 0.0000, 0.2696, 0.0000, 0.3854, 0.1379, 0.2452, 0.1252,
         0.1132, 0.0434, 0.0000, 0.4872, 0.1672, 0.3276, 0.0000, 0.2342, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)
After ReLU
tensor([[0.0000, 0.4028, 0.0000, 0.5694, 0.0000, 0.0000, 0.3374, 0.3942, 0.5255,
         0.1611, 0.1811, 0.0444, 0.4902, 0.6375, 0.1585, 0.0000, 0.1575, 0.0000,
         0.0000, 0.1211],
        [0.0000, 0.2777, 0.0000, 0.3033, 0.0000, 0.1457, 0.4572, 0.6595, 0.7586,
         0.3265, 0.0000, 0.0000, 0.6714, 0.6085, 0.2807, 0.0544, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.

nn.Sequential

nn.Sequential 순서를 갖는 모듈의 컨테이너이다.

데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달된다.

순차 컨테이너를 사용하여 아래의 seq_modules과 같은 신경망을 빠르게 만들 수 있다.

In [20]:
seq_modules = nn.Sequential(
    flatten, # nn.Flatten 3, 28, 28 -> 3, 28*28
    layer1, # nn.Linear 3, 28*28 ->
    nn.ReLU(), 
    nn.Linear(20,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[ 0.0906,  0.3778, -0.0954, -0.2084,  0.1566, -0.1968, -0.0596, -0.1879,
         -0.1914,  0.0277],
        [-0.0925,  0.2887,  0.1030, -0.1780,  0.1468, -0.2322, -0.1520, -0.2773,
         -0.1834,  0.0849],
        [ 0.0867,  0.2833,  0.0787, -0.2091, -0.0080, -0.1986, -0.1048, -0.0694,
         -0.0576, -0.0616]], grad_fn=<AddmmBackward>)

nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 logits를 반환한다.

logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1]범위로 비례하여 scale된다.

dim 매개변수는 값의 합이 1이 된다는 차원을 나타낸다.

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1107, 0.1475, 0.0919, 0.0821, 0.1183, 0.0830, 0.0953, 0.0838, 0.0835,
         0.1040],
        [0.0942, 0.1379, 0.1145, 0.0865, 0.1196, 0.0819, 0.0887, 0.0783, 0.0860,
         0.1125],
        [0.1108, 0.1349, 0.1099, 0.0824, 0.1008, 0.0833, 0.0915, 0.0948, 0.0959,
         0.0955]], grad_fn=<SoftmaxBackward>)

# 모델 매개변수

신경망 내부의 많은 레이어들은 매개변수화된다.

즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.

nn.Module을 상속하면 모델 객체 내부의 필드들이 자동으로 track되며,

모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수들을 순회하며, 매개변수의 크기와 값을 출력한다.

In [25]:
print("Model structure")
print(model)

for name, param in model.named_parameters():
    # print(param)
    # print(param[:2])
    print(f"Layer : {name} | size {param.size()} | values : {param[:2]} \n")

Model structure
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
Layer : linear_relu_stack.0.weight | size torch.Size([512, 784]) | values : tensor([[ 0.0261, -0.0253, -0.0081,  ..., -0.0139,  0.0025,  0.0233],
        [ 0.0144,  0.0136, -0.0313,  ...,  0.0174, -0.0203,  0.0298]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.0.bias | size torch.Size([512]) | values : tensor([0.0173, 0.0126], device='cuda:0', grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.2.weight | size torch.Size([512, 512]) | values : tensor([[-0.0230, -0.0294, -0.0041,  ...,  0.0326, -0.0242, -0.0296],
        [ 0.0266, -0.0211, -0.0200,  ...,  0.0442,  0.0242, -0.0016]],
       device='cuda:0', gr